In [8]:
import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

# ==============================================================================
# 1. GENERACIÓN DE DATOS SIMULADOS (Para que el ejemplo funcione sin tus CSVs)
# ==============================================================================
def generar_datos_demo():
    # Datos para la Curva de Trade-Off (Gráfico 1)
    flota = np.arange(50, 120, 2)
    servicio = 100 * (1 - np.exp(-(flota - 40)/10))
    servicio = np.clip(servicio, 80, 100)
    ociosidad = (flota - 58) * 0.8
    ociosidad = np.clip(ociosidad, 0, 50)
    
    df_curva = pd.DataFrame({
        'Flota_Total': flota,
        'Nivel_Servicio': servicio,
        'Tasa_Ociosidad': ociosidad
    })
    
    datos_optimos = {
        'Flota_Total_Recomendada': 58,
        'Nivel_Servicio_Logrado': 98.2,
        'Tasa_Ociosidad': 17.2
    }

    # Datos para Kilómetros (Gráfico 2 - Matplotlib original)
    df_km = pd.DataFrame({
        'Periodo': [12, 13, 14, 15, 17, 20, 29, 20], # Periodos como en tu foto
        'Distancia': [75000, 130000, 78000, 150000, 270000, 210000, 220000, 80000]
    })

    # Datos para Buffer (Gráfico 3 - Altair original)
    rutas = [f"Ruta {i}" for i in range(1, 21)]
    costos = np.random.randint(10000, 60000, 20)
    costos = sorted(costos, reverse=True)
    estrategias = ['MANTENER FLOTA (Wait)'] * 15 + ['MOVER EN VACÍO / CONSOLIDAR'] * 5
    
    df_buffer = pd.DataFrame({
        'Ruta': rutas,
        'Costo_Mensual_Buffer': costos,
        'Estrategia_Sugerida': estrategias
    })

    return df_curva, datos_optimos, df_km, df_buffer

# Cargamos los datos
df_curva, datos_optimos, df_km, df_buffer = generar_datos_demo()

# ==============================================================================
# 2. CONFIGURACIÓN DE LA APP Y ESTILOS
# ==============================================================================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "G-Logistics Dashboard"

# Paleta de colores extraída de tus imágenes
COLORS = {
    "orange": "#ec6c25",
    "dark_blue": "#2c3e50",
    "bg_light": "#f4f6f9",
    "text_grey": "#6c757d",
    "card_grey": "#868d8e"
}

# Estilo para el sidebar
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "1rem",
    "background-color": "#f8f9fa",
    "box-shadow": "2px 0 5px rgba(0,0,0,0.1)",
    "z-index": 100
}

# Estilo para el contenido principal
CONTENT_STYLE = {
    "margin-left": "17rem",
    "padding": "1rem",
    "background-color": COLORS["bg_light"],
    "min-height": "100vh"
}

# ==============================================================================
# 3. COMPONENTES DEL LAYOUT
# ==============================================================================

# --- Sidebar ---
sidebar = html.Div([
    # Logo (Simulado con texto/estilo)
    html.Div([
        html.H1("G", style={"font-size": "4rem", "color": COLORS["orange"], "font-weight": "900", "display": "inline"}),
        html.Span("->", style={"font-size": "3rem", "color": COLORS["dark_blue"], "font-weight": "bold"})
    ], className="text-center mb-4"),
    
    dbc.Nav([
        dbc.NavLink([html.I(className="bi bi-speedometer2 me-2"), "Inicio"], href="/", active="exact"),
        dbc.NavLink([html.I(className="bi bi-grid me-2"), "Dashboard"], href="/dashboard", active="exact"),
    ], vertical=True, pills=True, className="mb-4"),

    # Search Box (Simulando la imagen 2)
    html.Div([
        html.Label("Buscar Ruta/Cliente", className="text-muted small"),
        dcc.Dropdown(
            id='ruta-dropdown',
            options=[
                {'label': 'BB PACABTUN/BB PONIENTE', 'value': 'BB_PACABTUN'},
                {'label': 'BB CANCUN PLANTA', 'value': 'BB_CANCUN'},
            ],
            value='BB_PACABTUN',
            placeholder="Search...",
            style={"font-size": "0.9rem"}
        )
    ], className="mb-4 bg-light p-2 rounded"),

    # User Profile (Bottom)
    html.Div([
        html.Hr(),
        dbc.Row([
            dbc.Col(html.Div(style={"width": "30px", "height": "30px", "background-color": COLORS["dark_blue"], "border-radius": "50%"}), width=3),
            dbc.Col(html.P("User1", className="mb-0"), width=9)
        ], align="center")
    ], style={"position": "absolute", "bottom": "1rem", "width": "85%"})

], style=SIDEBAR_STYLE)

# --- Tarjetas KPI (Top Row) ---
def create_kpi_card(title, value, bg_color=COLORS["card_grey"]):
    return dbc.Card([
        dbc.CardBody([
            html.H6(title, className="card-title text-white small"),
            html.H3(value, className="card-text text-white fw-bold")
        ])
    ], style={"background-color": bg_color, "border": "none", "border-radius": "15px"}, className="shadow-sm h-100")

# --- Gráficos (Conversión a Plotly) ---

def fig_tradeoff():
    # Recreando tu lógica de Matplotlib en Plotly
    fig = go.Figure()

    # Eje Izquierdo: Nivel de Servicio (Azul)
    fig.add_trace(go.Scatter(
        x=df_curva['Flota_Total'], y=df_curva['Nivel_Servicio'],
        name='Nivel Servicio', mode='lines+markers',
        line=dict(color='#1f77b4', width=3), marker=dict(size=8)
    ))

    # Eje Derecho: Ociosidad (Naranja)
    fig.add_trace(go.Scatter(
        x=df_curva['Flota_Total'], y=df_curva['Tasa_Ociosidad'],
        name='Ociosidad', mode='lines+markers',
        yaxis='y2', line=dict(color='#ff7f0e', dash='dot', width=3), marker=dict(symbol='x', size=8)
    ))

    # Línea de Meta (Verde)
    fig.add_shape(type="line", x0=df_curva['Flota_Total'].min(), x1=df_curva['Flota_Total'].max(),
                  y0=98, y1=98, line=dict(color="green", dash="dash"), name="Meta 98%")

    # Línea vertical del óptimo
    fig.add_shape(type="line", x0=datos_optimos['Flota_Total_Recomendada'], x1=datos_optimos['Flota_Total_Recomendada'],
                  y0=80, y1=100, line=dict(color="gray", width=1), yref='y')

    # Anotación (Box de información)
    fig.add_annotation(
        x=datos_optimos['Flota_Total_Recomendada'], y=88,
        text=f"SOLUCIÓN ÓPTIMA:<br>{datos_optimos['Flota_Total_Recomendada']} Camiones<br>NS: {datos_optimos['Nivel_Servicio_Logrado']}%<br>Ociosidad: {datos_optimos['Tasa_Ociosidad']}%",
        showarrow=True, arrowhead=1, ax=40, ay=40,
        bgcolor="white", bordercolor="black", borderwidth=1
    )

    # --- ESTA ES LA PARTE QUE DABA ERROR CORREGIDA ---
    fig.update_layout(
        title="Trade - Off Servicio vs. Costo",
        xaxis_title="Tamaño de Flota Total (Unidades)",
        
        # Corrección: titlefont ya no se usa, ahora es font dentro de title
        yaxis=dict(
            title=dict(text="Nivel de Servicio (%)", font=dict(color="#1f77b4")),
            range=[80, 102]
        ),
        
        # Corrección: titlefont ya no se usa aquí tampoco
        yaxis2=dict(
            title=dict(text="Tasa de Ociosidad (%)", font=dict(color="#ff7f0e")),
            overlaying="y", 
            side="right", 
            range=[0, 50]
        ),
        
        legend=dict(x=0.01, y=0.99),
        margin=dict(l=40, r=40, t=40, b=40),
        height=350,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        hovermode="x unified"
    )
    return fig

# ==============================================================================
# 4. LAYOUT PRINCIPAL
# ==============================================================================

content = html.Div([
    # Header Breadcrumb
    dbc.Row([
        dbc.Col([
            html.Div([
                html.I(className="bi bi-layout-sidebar me-3"),
                html.Span("Inicio / ", className="text-muted"),
                html.Span("BB PACABTUN/BB PONIENTE", className="fw-bold")
            ], className="d-flex align-items-center mb-4")
        ]),
        dbc.Col([
            # Iconos header derecha
             html.Div([
                html.I(className="bi bi-sun me-3"),
                html.I(className="bi bi-clock-history me-3"),
                html.I(className="bi bi-bell")
            ], className="d-flex justify-content-end text-muted")
        ])
    ]),

    # KPI Row
    dbc.Row([
        dbc.Col(create_kpi_card("Tasa de Ociosidad", "17.24%", bg_color=COLORS["orange"]), md=3),
        dbc.Col(create_kpi_card("Costo Combustible", "$45.83"), md=3),
        dbc.Col(create_kpi_card("CPK", "$43.33", bg_color=COLORS["orange"]), md=3),
        dbc.Col(create_kpi_card("Nivel de Servicio", "98%"), md=3),
    ], className="mb-4"),

    # Middle Section: Cluster Info + Main Graph
    dbc.Row([
        # Panel Izquierdo (Info Critica)
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                    html.H6("LA RUTA PERTENECE AL CLÚSTER", className="small text-muted mb-3"),
                    html.Div([
                        html.Div("CRÍTICO", className="d-flex align-items-center justify-content-center fw-bold",
                                 style={"width": "100px", "height": "100px", "background-color": COLORS["orange"], 
                                        "border-radius": "50%", "margin": "0 auto", "color": "black"})
                    ], className="text-center mb-3"),
                    html.P("Estrategia: Mantener en ruta 2 días", className="text-center fw-bold small")
                ])
            ], className="h-100 shadow-sm border-0")
        ], md=3),

        # Panel Derecho (Gráfico Trade-off)
        dbc.Col([
            dbc.Card([
                dbc.CardBody([
                     dcc.Graph(figure=fig_tradeoff(), config={'displayModeBar': False})
                ])
            ], className="h-100 shadow-sm border-0")
        ], md=9),
    ], className="mb-4"),

    # Bottom Section: KM Graph + Buffer Graph
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardBody(dcc.Graph(figure=fig_km(), config={'displayModeBar': False}))
            ], className="shadow-sm border-0")
        ], md=6),
        dbc.Col([
            dbc.Card([
                dbc.CardBody(dcc.Graph(figure=fig_buffer(), config={'displayModeBar': False}))
            ], className="shadow-sm border-0")
        ], md=6),
    ])

], style=CONTENT_STYLE)

app.layout = html.Div([sidebar, content])

# ==============================================================================
# 5. EJECUCIÓN
# ==============================================================================
if __name__ == "__main__":
    app.run(debug=True)

[2025-12-04 22:17:29,797] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/dash/dash.py", line 1477, in _prepare_callback
    cb = self.callback_map[output]
         ~~~~~~~~~~~~~~~~~^^^^^^^^
KeyError: 'page-content.children'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Ve